## Preamble

In [2]:
import pandas as pd
import numpy as np

%matplotlib inline

In [3]:
drinks = pd.read_csv("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Raw/Drinks.csv", sep=",")

## Drinks

We start with out first dataset and moving forward, I will try to check the data quality of our given data. I will try to check if there are any missing values and duplicates. Later on, I will try to transform the datatypes to ones I would like to work with.

In [4]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,NaN
1,Albania,89,132,54,NaN
2,Algeria,25,0,14,NaN
3,Andorra,245,138,312,NaN
4,Angola,217,57,45,NaN


In [5]:
drinks.isnull().sum()

country                           0
beer_servings                     0
spirit_servings                   0
wine_servings                     0
total_litres_of_pure_alcohol    193
dtype: int64

In [6]:
drinks.shape, drinks.drop_duplicates().shape

((193, 5), (193, 5))

In [7]:
drinks.dtypes

country                          object
beer_servings                    object
spirit_servings                  object
wine_servings                    object
total_litres_of_pure_alcohol    float64
dtype: object

## Datatype transformations

Changing the datatypes of servings columns didn't work. There must be non numerical values (indeed there are) in our observations. We can coerce them as below and we can work on how to handle the missing values later on. 

In [8]:
drinks["beer_servings"] = pd.to_numeric(drinks["beer_servings"], errors = "coerce")

In [9]:
drinks["spirit_servings"] = pd.to_numeric(drinks["spirit_servings"], errors = "coerce")

In [10]:
drinks["wine_servings"] = pd.to_numeric(drinks["wine_servings"], errors = "coerce")

There seems to be question marks in the data instead of null or numeric observations. Missing observations and their correspondent countries can be seen below.

In [11]:
drinks[drinks["beer_servings"].isnull()]["country"],drinks[drinks["spirit_servings"].isnull()]["country"], drinks[drinks["wine_servings"].isnull()]["country"] 

(99    Macedonia
 Name: country, dtype: object,
 46    Denmark
 Name: country, dtype: object,
 11    Bahamas
 Name: country, dtype: object)

Maybe it is best to handle these null values with income group level mean or median. For instance, taking the average of spirit servings of all countries and apply it for Denmark's spirit consumption could be misleading. It would be best to take a look at high income countries' or a regional average. To do that, we are moving to another dataset we have.

I will start with some data wrangling and eventually merge our datasets into one.

## Countries

In [12]:
countries = pd.ExcelFile("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Raw/CountriesOfTheWorld.xls")
countries = pd.read_excel(countries, header = None)

In [13]:
countries = countries[3:]
countries = countries.drop([4])

In [14]:
countries.columns = countries.iloc[0]

In [15]:
countries = countries[1:] 

In [16]:
countries = countries.reset_index()

In [17]:
countries = countries.drop("index", axis = 1)

In [18]:
countries.rename(columns = {"Country" : "country"}, inplace = True)

In [19]:
countries.shape, countries.drop_duplicates().shape

((227, 16), (227, 16))

There seems to be whitespaces among our country variable. That makes it hard to merge our two dataset together. We should correct this in order to continue with our analysis.

In [20]:
countries.loc[:, "country"] = countries.loc[:, "country"].str.strip()
drinks.loc[:, "country"] = drinks.loc[:, "country"].str.strip()

In [21]:
combined = countries.merge(drinks, on = "country")

In [22]:
combined.shape, drinks.shape

((174, 20), (193, 5))

It seems that country names are different in each dataset. I will try to correct it manually and in order to do that, I will print countries that doesn't appear in either one of the datasets.

In [23]:
drinks[~drinks.country.isin(combined["country"])]

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
11,Bahamas,122.0,176.0,NaN,NaN
21,Bosnia-Herzegovina,76.0,173.0,8.0,NaN
28,Cabo Verde,144.0,56.0,16.0,NaN
32,Central African Republic,17.0,2.0,1.0,NaN
38,Congo,76.0,1.0,9.0,NaN
50,DR Congo,32.0,3.0,1.0,NaN
62,Gambia,8.0,0.0,1.0,NaN
110,Micronesia,62.0,50.0,18.0,NaN
114,Montenegro,31.0,114.0,128.0,NaN
117,Myanmar,5.0,1.0,0.0,NaN


In [24]:
countries[~countries.country.isin(combined["country"])]

3,country,Region,Population,Area,Pop. Density,Coastline,Net migration,Infant mortality,GDP,Literacy,Phones,Arable,Crops,Other,Birthrate,Deathrate
3,American Samoa,OCEANIA,57794,199,290.42,58.29,-20.71,9.27,8000,97,259.54,10,15,75,22.46,3.27
6,Anguilla,LATIN AMER. & CARIB,13477,102,132.13,59.8,10.76,21.03,8600,95,460.04,0,0,100,14.17,5.34
10,Aruba,LATIN AMER. & CARIB,71891,193,372.49,35.49,0,5.89,28000,97,516.06,10.53,0,89.47,11.03,6.68
14,"Bahamas, The",LATIN AMER. & CARIB,303770,13940,21.79,25.41,-2.2,25.21,16700,95.6,460.55,0.8,0.4,98.8,17.57,9.05
22,Bermuda,NORTHERN AMERICA,65773,53,1241,194.34,2.49,8.53,36000,98,851.41,20,0,80,11.4,7.74
25,Bosnia & Herzegovina,EASTERN EUROPE,4498976,51129,87.99,0.04,0.31,21.05,6100,NaN,215.36,13.6,2.96,83.44,8.77,8.27
28,British Virgin Is.,LATIN AMER. & CARIB,23098,153,150.97,52.29,10.01,18.05,16000,97.8,506.54,20,6.67,73.33,14.89,4.42
32,Burma,ASIA (EX. NEAR EAST),47382633,678500,69.83,0.28,-1.8,67.24,1800,85.3,10.05,15.19,0.97,83.84,17.91,9.83
37,Cape Verde,SUB-SAHARAN AFRICA,420979,4033,104.38,23.93,-12.07,47.77,1400,76.6,169.6,9.68,0.5,89.82,24.87,6.55
38,Cayman Islands,LATIN AMER. & CARIB,45436,262,173.42,61.07,18.75,8.19,35000,98,836.34,3.85,0,96.15,12.74,4.89


In [25]:
countries.iloc[14,0] = drinks.iloc[11,0]
countries.iloc[25,0] = drinks.iloc[21,0]
countries.iloc[37,0] = drinks.iloc[28,0]
countries.iloc[39,0] = drinks.iloc[32,0]
countries.iloc[46,0] = drinks.iloc[38,0]
countries.iloc[45,0] = drinks.iloc[50,0]
countries.iloc[73,0] = drinks.iloc[62,0]
countries.iloc[136,0] = drinks.iloc[110,0]
countries.iloc[32,0] = drinks.iloc[117,0]
countries.iloc[109,0] = drinks.iloc[127,0]
countries.iloc[169,0] = drinks.iloc[141,0]
countries.iloc[110,0] = drinks.iloc[157,0]
countries.iloc[172,0] = drinks.iloc[160,0]
countries.iloc[173,0] = drinks.iloc[161,0]
countries.iloc[175,0] = drinks.iloc[162,0]
countries.iloc[214,0] = drinks.iloc[185,0]
countries.iloc[58,0] = drinks.iloc[172,0]

In [26]:
combined = countries.merge(drinks, on = "country")

In [27]:
combined.shape, drinks.shape

((191, 20), (193, 5))

Not perfect but it is definitely better. We seem to be able to combine 191 countries with country information and alcohol consumption information. Now it is time to fill in the missing values we have find out for alcohol servings in the beginning of our analysis.

In [28]:
combined[combined["country"] == "Denmark"]["Region"], combined[combined["country"] == "Macedonia"]["Region"], combined[combined["country"] == "Bahamas"]["Region"]

(48    WESTERN EUROPE                     
 Name: Region, dtype: object,
 103    EASTERN EUROPE                     
 Name: Region, dtype: object,
 11    LATIN AMER. & CARIB    
 Name: Region, dtype: object)

In [29]:
combined.groupby(["Region"]).mean()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
Region,,,,
ASIA (EX. NEAR EAST),37.640000,54.200000,7.360000,NaN
BALTICS,282.666667,218.000000,59.000000,NaN
C.W. OF IND. STATES,83.250000,168.083333,33.416667,NaN
EASTERN EUROPE,237.272727,155.666667,129.750000,NaN
LATIN AMER. & CARIB,150.212121,148.757576,33.687500,NaN
NEAR EAST,30.928571,44.285714,14.071429,NaN
NORTHERN AFRICA,18.800000,2.600000,9.000000,NaN
NORTHERN AMERICA,244.500000,140.000000,92.000000,NaN
OCEANIA,83.133333,49.000000,37.533333,NaN


In [30]:
combined[combined["country"] == "Denmark"]

,country,Region,Population,Area,Pop. Density,Coastline,Net migration,Infant mortality,GDP,Literacy,Phones,Arable,Crops,Other,Birthrate,Deathrate,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
48,Denmark,WESTERN EUROPE,5450661,43094,126.48,16.97,2.48,4.56,31100,100,614.6,54.02,0.19,45.79,11.13,10.36,224.0,NaN,278.0,NaN


In [31]:
combined.iloc[48,17] = 92.047619

In [32]:
combined[combined["country"] == "Macedonia"]

,country,Region,Population,Area,Pop. Density,Coastline,Net migration,Infant mortality,GDP,Literacy,Phones,Arable,Crops,Other,Birthrate,Deathrate,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
103,Macedonia,EASTERN EUROPE,2050554,25333,80.94,0,-1.45,10.09,6700,NaN,260.03,22.26,1.81,75.93,12.02,8.77,NaN,27.0,86.0,NaN


In [33]:
combined.iloc[103,16] = 253.400000

In [34]:
combined[combined["country"] == "Bahamas"]

,country,Region,Population,Area,Pop. Density,Coastline,Net migration,Infant mortality,GDP,Literacy,Phones,Arable,Crops,Other,Birthrate,Deathrate,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
11,Bahamas,LATIN AMER. & CARIB,303770,13940,21.79,25.41,-2.2,25.21,16700,95.6,460.55,0.8,0.4,98.8,17.57,9.05,122.0,176.0,NaN,NaN


In [35]:
combined.iloc[11,18] = 33.241379

Okay, we have merged country information and their alcoholic berage consumption and handled our missing values for Bahamas, Macedonia and Denmark. We lost Niue and Montenegro in the process but it could be expected and not a huge loss since we end up with 191 observations out of 193. Now we should calculate the total litres of pure alcohol per serving we have given, join life expectancy data and tidy all of our variables a little.

In [36]:
combined["total_litres_of_pure_alcohol"] = combined["beer_servings"]*12*0.0295*0.05 + combined["wine_servings"]*5*0.0295*0.12 + combined["spirit_servings"]*1.5*0.0295*0.40 

In [37]:
combined.dtypes

country                          object
Region                           object
Population                       object
Area                             object
Pop. Density                     object
Coastline                        object
Net migration                    object
Infant mortality                 object
GDP                              object
Literacy                         object
Phones                           object
Arable                           object
Crops                            object
Other                            object
Birthrate                        object
Deathrate                        object
beer_servings                   float64
spirit_servings                 float64
wine_servings                   float64
total_litres_of_pure_alcohol    float64
dtype: object

In [38]:
combined["Population"] = pd.to_numeric(combined["Population"], errors = "coerce")
combined["Area"] = pd.to_numeric(combined["Area"], errors = "coerce")
combined["Pop. Density"] = pd.to_numeric(combined["Pop. Density"], errors = "coerce")
combined["Coastline"] = pd.to_numeric(combined["Coastline"], errors = "coerce")
combined["Net migration"] = pd.to_numeric(combined["Net migration"], errors = "coerce")
combined["Infant mortality"] = pd.to_numeric(combined["Infant mortality"], errors = "coerce")
combined["GDP"] = pd.to_numeric(combined["GDP"], errors = "coerce")
combined["Literacy"] = pd.to_numeric(combined["Literacy"], errors = "coerce")
combined["Phones"] = pd.to_numeric(combined["Phones"], errors = "coerce")
combined["Arable"] = pd.to_numeric(combined["Arable"], errors = "coerce")
combined["Crops"] = pd.to_numeric(combined["Crops"], errors = "coerce")
combined["Other"] = pd.to_numeric(combined["Other"], errors = "coerce")
combined["Birthrate"] = pd.to_numeric(combined["Birthrate"], errors = "coerce")
combined["Deathrate"] = pd.to_numeric(combined["Deathrate"], errors = "coerce")

In [39]:
combined.isna().sum()

country                         0
Region                          0
Population                      0
Area                            0
Pop. Density                    0
Coastline                       0
Net migration                   1
Infant mortality                1
GDP                             0
Literacy                        7
Phones                          2
Arable                          0
Crops                           0
Other                           0
Birthrate                       1
Deathrate                       2
beer_servings                   0
spirit_servings                 0
wine_servings                   0
total_litres_of_pure_alcohol    0
dtype: int64

Data types looks better now. There are null values here and there but to be honest, I mostly care about GDP and it has no null values at this point. We will see if we need to handle others but for the time being we can leave them as it is.

## Life Expectancy

In [40]:
life = pd.read_csv("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Raw/LifeExpectancy.csv", sep=",")

In [41]:
life.shape

(6408, 16)

In [42]:
life.columns

Index(['GhoCode', 'GhoDisplay', 'PublishStateCode', 'PublishStateDisplay',
       'YearCode', 'YearDisplay', 'RegionCode', 'RegionDisplay',
       'WorldBankIncomeGroupGroupCode', 'WorldBankIncomeGroupDisplay',
       'CountryCode', 'CountryDisplay', 'SexCode', 'SexDisplay',
       'DisplayValue', 'Numeric'],
      dtype='object')

In [43]:
life['YearCode'].unique(), life['SexDisplay'].unique(), life['GhoDisplay'].unique()

(array([2013, 2012, 2000, 1990]),
 array(['Both sexes', 'Female', 'Male'], dtype=object),
 array(['Life expectancy at birth (years)',
        'Healthy life expectancy (HALE) at birth (years)',
        'Life expectancy at age 60 (years)'], dtype=object))

There are 3 distinct data on 4 distint years and on 2 sexes on our dataset. We are asked to check the most current data so I will opt for using 2013 values and I also choose to include only the average value of both sexes since I don't have extra information on sexes' alcohol consumption patterns. I will keep 3 different measures of life expectancy for the sake of easy reproducablity, even though I highly doubt the results will change. 

In [44]:
most_life = life[life["GhoDisplay"] == 'Life expectancy at birth (years)']
healthy_life = life[life["GhoDisplay"] == 'Healthy life expectancy (HALE) at birth (years)']
after_life = life[life["GhoDisplay"] == 'Life expectancy at age 60 (years)']

In [45]:
most_life = most_life[{"YearCode", "CountryDisplay", "SexDisplay", "Numeric"}]
healthy_life = healthy_life[{"YearCode", "CountryDisplay", "SexDisplay", "Numeric"}]
after_life = after_life[{"YearCode", "CountryDisplay", "SexDisplay", "Numeric"}]

In [46]:
most_life = most_life[(most_life["SexDisplay"] == "Both sexes") & (most_life["YearCode"] == 2013)]
healthy_life = healthy_life[(healthy_life["SexDisplay"] == "Both sexes") & (healthy_life["YearCode"] == 2013)]
after_life = after_life[(after_life["SexDisplay"] == "Both sexes") & (after_life["YearCode"] == 2013)]


In [47]:
most_life = most_life.reset_index().iloc[:, 1:5]
healthy_life = healthy_life.reset_index().iloc[:, 1:5]
after_life = after_life.reset_index().iloc[:, 1:5]

In [48]:
most_life.shape, healthy_life.shape, after_life.shape

((194, 4), (194, 4), (194, 4))

In [49]:
most_life.columns

Index(['YearCode', 'SexDisplay', 'CountryDisplay', 'Numeric'], dtype='object')

In [50]:
combined.merge(most_life, left_on = "country", right_on = "CountryDisplay").shape

(165, 24)

It seems that country names are again different in our datasets. I will again correct them manually.

In [51]:
most_life[~most_life.CountryDisplay.isin(combined["country"])]["CountryDisplay"]

5                                    Antigua and Barbuda
20                      Bolivia (Plurinational State of)
21                                Bosnia and Herzegovina
24                                     Brunei Darussalam
41                                         Côte d'Ivoire
46                 Democratic People's Republic of Korea
47                      Democratic Republic of the Congo
71                                         Guinea_Bissau
79                            Iran (Islamic Republic of)
92                      Lao People's Democratic Republic
110                     Micronesia (Federated States of)
113                                           Montenegro
125                                                 Niue
138                                    Republic of Korea
139                                  Republic of Moldova
143                                Saint Kitts and Nevis
144                                          Saint Lucia
145                     Saint V

In [52]:
combined[~combined.country.isin(most_life["CountryDisplay"])]["country"]

5                 Antigua & Barbuda
20                          Bolivia
21               Bosnia-Herzegovina
24                           Brunei
39                         DR Congo
43                    Cote d'Ivoire
52                      Timor-Leste
72                    Guinea-Bissau
80                             Iran
91                      North Korea
92                      South Korea
95                             Laos
103                       Macedonia
114                      Micronesia
115                         Moldova
143               St. Kitts & Nevis
144                       St. Lucia
145    St. Vincent & the Grenadines
148             Sao Tome & Principe
167                           Syria
169                        Tanzania
173               Trinidad & Tobago
181                  United Kingdom
182                             USA
186                       Venezuela
187                         Vietnam
Name: country, dtype: object

In [53]:
combined.iloc[5,0] = most_life.iloc[5,0]
combined.iloc[20,0] = most_life.iloc[20,0]
combined.iloc[21,0] = most_life.iloc[21,0]
combined.iloc[24,0] = most_life.iloc[24,0]
combined.iloc[39,0] = most_life.iloc[47,0]
combined.iloc[43,0] = most_life.iloc[41,0]
combined.iloc[52,0] = most_life.iloc[172,0]
combined.iloc[72,0] = most_life.iloc[71,0]
combined.iloc[80,0] = most_life.iloc[79,0]
combined.iloc[91,0] = most_life.iloc[46,0]
combined.iloc[92,0] = most_life.iloc[138,0]
combined.iloc[95,0] = most_life.iloc[92,0]
combined.iloc[103,0] = most_life.iloc[171,0]
combined.iloc[114,0] = most_life.iloc[110,0]
combined.iloc[115,0] = most_life.iloc[139,0]
combined.iloc[143,0] = most_life.iloc[143,0]
combined.iloc[144,0] = most_life.iloc[144,0]
combined.iloc[145,0] = most_life.iloc[145,0]
combined.iloc[148,0] = most_life.iloc[148,0]
combined.iloc[167,0] = most_life.iloc[168,0]
combined.iloc[169,0] = most_life.iloc[184,0]
combined.iloc[173,0] = most_life.iloc[175,0]
combined.iloc[181,0] = most_life.iloc[183,0]
combined.iloc[182,0] = most_life.iloc[185,0]
combined.iloc[186,0] = most_life.iloc[189,0]
combined.iloc[187,0] = most_life.iloc[190,0]


In [54]:
combined.merge(most_life, left_on = "country", right_on = "CountryDisplay").shape

(165, 24)

In [55]:
most_combined = combined.merge(most_life, left_on = "country", right_on = "CountryDisplay")
healthy_combined = combined.merge(healthy_life, left_on = "country", right_on = "CountryDisplay")
after_combined = combined.merge(after_life, left_on = "country", right_on = "CountryDisplay")


In [109]:
most_combined.to_csv("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Clean/most_combined.csv", index = False)
healthy_combined.to_csv("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Clean/healthy_combined.csv", index = False)
after_combined.to_csv("/home/ozzy/Documents/CEU/MPDS/Homework/Data/Clean/after_combined.csv", index = False)